In [10]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.svm import SVR

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from category_encoders import LeaveOneOutEncoder
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import SVR


In [2]:
raw_data = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/epi_r.csv')

In [3]:
raw_data.head()

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
raw_data2=raw_data.drop(columns=['calories', 'protein', 'fat', 'sodium']).dropna()
X=raw_data2.drop(columns=['rating', 'title']).sample(frac=0.3, replace=True, random_state=1)
y=raw_data2.rating.sample(frac=0.3, replace=True, random_state=1)

In [5]:
y=y.where(y>2,0)
y=y.where(y==0,1)
y.value_counts()

1.0    5361
0.0     655
Name: rating, dtype: int64

In [6]:
y.nunique()

2

In [7]:
drop_cols=[]
for col in X:
    if X[col].nunique()==1:
        drop_cols+=[col]
X=X.drop(columns=drop_cols)

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6016 entries, 235 to 13130
Columns: 602 entries, #cakeweek to turkey
dtypes: float64(602)
memory usage: 27.7 MB


In [9]:
X_new = SelectKBest(chi2, k=30).fit_transform(X, y)

In [15]:
from sklearn.svm import SVC
svc=SVC()
svc.fit(X_new, y)
svc.score(X_new,y)

0.9150598404255319

In [16]:
from sklearn.model_selection import cross_val_score
cross_val_score(svc, X_new, y, cv=3)

array([0.90578265, 0.90473815, 0.8967581 ])